In [3]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [2]:
# 讀取資料
df = pd.read_csv('./final/final/xy.csv')
df

,airline,Event Date,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count,Count[-1:+1],...,Corporation size,LEV,Current ratio,EPS,Adjusted Event Date,"SCAAR[-1,0]","SCAAR[-1,+1]",SAAR_t-1,SAAR_t0,SAAR_t1
0,china,2019/4/16,Label_4,11.0,-0.021074,0.424542,-0.392421,5.0,6.0,14.0,...,12239.0,0.7948,0.6787,-0.22,2019/4/16,1.340496,1.240263,1.331280,0.009216,-0.100233
1,china,2019/4/23,Label_5,6.0,0.171785,0.295388,-0.446230,5.0,1.0,11.0,...,12240.0,0.7948,0.6787,-0.22,2019/4/23,0.136689,-0.528828,0.076065,0.060624,-0.665517
2,china,2019/4/24,Label_7,4.0,0.212743,0.348105,-0.193345,3.0,1.0,8.0,...,12241.0,0.7948,0.6787,-0.22,2019/4/24,-0.669631,-0.179595,0.044270,-0.713901,0.490036
3,china,2019/4/25,Label_1,10.0,-0.230418,0.068513,-0.305150,2.0,8.0,15.0,...,12242.0,0.7948,0.6787,-0.22,2019/4/25,-0.154368,0.643694,-0.709882,0.555513,0.798063
4,china,2019/5/3,Label_7,4.0,0.163797,0.504764,-0.177171,2.0,2.0,4.0,...,12243.0,0.7948,0.6787,-0.22,2019/5/3,-0.358229,-1.596006,-0.705435,0.347207,-1.237778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,tiger,2024/10/6,Label_3,5.0,-0.602064,0.000000,-0.602064,0.0,5.0,5.0,...,795.0,0.6591,1.6242,6.16,2024/10/7,-0.269476,-0.291187,-0.951216,0.681741,-0.021712
165,tiger,2024/10/25,Label_1,5.0,-0.125567,0.142503,-0.304279,2.0,3.0,5.0,...,795.0,0.6591,1.6242,6.16,2024/10/25,0.957682,0.678913,0.715947,0.241735,-0.278768
166,tiger,2024/10/25,Label_7,2.0,-0.001633,0.378198,-0.381463,1.0,1.0,2.0,...,795.0,0.6591,1.6242,6.16,2024/10/25,0.957682,0.678913,0.715947,0.241735,-0.278768
167,tiger,2024/12/2,Label_3,4.0,-0.143172,0.170369,-0.247685,1.0,3.0,4.0,...,795.0,0.6591,1.6242,6.16,2024/12/2,-0.695964,1.922002,-0.197526,-0.498437,2.617966


In [5]:
# 自定分組條件
def map_group(dept):
    if dept in ['Label_1','Label_2','Label_3']:
        return 'Group_1'
    elif dept in ['Label_4','Label_5','Label_6']:
        return 'Group_2'
    else:
        return 'Group_3'


df['Group'] = df['Label'].apply(map_group)

df


,airline,Event Date,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count,Count[-1:+1],...,LEV,Current ratio,EPS,Adjusted Event Date,"SCAAR[-1,0]","SCAAR[-1,+1]",SAAR_t-1,SAAR_t0,SAAR_t1,Group
0,china,2019/4/16,Label_4,11.0,-0.021074,0.424542,-0.392421,5.0,6.0,14.0,...,0.7948,0.6787,-0.22,2019/4/16,1.340496,1.240263,1.331280,0.009216,-0.100233,Group_2
1,china,2019/4/23,Label_5,6.0,0.171785,0.295388,-0.446230,5.0,1.0,11.0,...,0.7948,0.6787,-0.22,2019/4/23,0.136689,-0.528828,0.076065,0.060624,-0.665517,Group_2
2,china,2019/4/24,Label_7,4.0,0.212743,0.348105,-0.193345,3.0,1.0,8.0,...,0.7948,0.6787,-0.22,2019/4/24,-0.669631,-0.179595,0.044270,-0.713901,0.490036,Group_3
3,china,2019/4/25,Label_1,10.0,-0.230418,0.068513,-0.305150,2.0,8.0,15.0,...,0.7948,0.6787,-0.22,2019/4/25,-0.154368,0.643694,-0.709882,0.555513,0.798063,Group_1
4,china,2019/5/3,Label_7,4.0,0.163797,0.504764,-0.177171,2.0,2.0,4.0,...,0.7948,0.6787,-0.22,2019/5/3,-0.358229,-1.596006,-0.705435,0.347207,-1.237778,Group_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,tiger,2024/10/6,Label_3,5.0,-0.602064,0.000000,-0.602064,0.0,5.0,5.0,...,0.6591,1.6242,6.16,2024/10/7,-0.269476,-0.291187,-0.951216,0.681741,-0.021712,Group_1
165,tiger,2024/10/25,Label_1,5.0,-0.125567,0.142503,-0.304279,2.0,3.0,5.0,...,0.6591,1.6242,6.16,2024/10/25,0.957682,0.678913,0.715947,0.241735,-0.278768,Group_1
166,tiger,2024/10/25,Label_7,2.0,-0.001633,0.378198,-0.381463,1.0,1.0,2.0,...,0.6591,1.6242,6.16,2024/10/25,0.957682,0.678913,0.715947,0.241735,-0.278768,Group_3
167,tiger,2024/12/2,Label_3,4.0,-0.143172,0.170369,-0.247685,1.0,3.0,4.0,...,0.6591,1.6242,6.16,2024/12/2,-0.695964,1.922002,-0.197526,-0.498437,2.617966,Group_1


In [8]:
# 確認 Label 欄位存在（替換成你實際的欄位名稱）
label_col = 'Group'
saar_col = 'SCAAR[-1,+1]'

# 分群 t 檢定
labels = df[label_col].dropna().unique()

print(f"{'Label':<10} {'Mean':>10} {'t-stat':>10} {'p-value':>10} {'n':>5} {'Signif.':>8}")
print("-" * 60)

for label in labels:
    group = df[df[label_col] == label][saar_col].dropna()
    if len(group) < 2:
        continue  # 忽略樣本數太少的群組
    
    mean = group.mean()
    t_stat, p_val = stats.ttest_1samp(group, 0)

    # 顯著性標記
    if p_val < 0.001:
        stars = '***'
    elif p_val < 0.01:
        stars = '**'
    elif p_val < 0.05:
        stars = '*'
    else:
        stars = ''
    
    print(f"{label:<10} {mean:>10.4f} {t_stat:>10.3f} {p_val:>10.4f} {len(group):>5} {stars:>8}")


Label            Mean     t-stat    p-value     n  Signif.
------------------------------------------------------------
Group_2        0.1577      0.624     0.5346    66         
Group_3       -0.1312     -0.403     0.6902    30         
Group_1        0.0916      0.370     0.7128    72         
